In [ ]:
import tensorflow as tf
import pickle
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Layer
from tensorflow.keras.utils import to_categorical
import numpy as np

In [ ]:
def load_cifar10_batch(file):
    with open(file, 'rb') as f:
        dict = pickle.load(f, encoding='bytes')
        X = dict[b'data']
        Y = dict[b'labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0, 2, 3, 1).astype("float")
        Y = np.array(Y)
        return X, Y

# Load all training batches
X_train = []
Y_train = []
for i in range(1, 6):
    X_batch, Y_batch = load_cifar10_batch(f'\cifar-10-batches-py\data_batch_{i}')
    X_train.append(X_batch)
    Y_train.append(Y_batch)

X_train = np.concatenate(X_train, axis=0)
Y_train = np.concatenate(Y_train, axis=0)

# Load test batch
X_test, Y_test = load_cifar10_batch('cifar-10-batches-py\test_batch')

# Normalize pixel values to be between 0 and 1
X_train, X_test = X_train / 255.0, X_test / 255.0

# Convert class vectors to binary class matrices (one-hot encoding)
Y_train = to_categorical(Y_train, 10)
Y_test = to_categorical(Y_test, 10)

We have to implement a Local Response Normalization (LRN) Layer, which is used in AlexNet but is not predefined in keras.

In [ ]:
class LRN(Layer):
    def __init__(self, **kwargs):
        super(LRN, self).__init__(**kwargs)

    def call(self, x):
        return tf.nn.local_response_normalization(x, depth_radius=5, alpha=0.0001, beta=0.75, bias=2.0)

In [ ]:
alexnet = Sequential()

# 1st Convolutional Layer
alexnet.add(Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11), strides=(4,4), padding='valid', activation='relu'))
alexnet.add(BatchNormalization())
alexnet.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
alexnet.add(LRN())

# 2nd Convolutional Layer
alexnet.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same', activation='relu'))
alexnet.add(BatchNormalization())
alexnet.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
alexnet.add(LRN())

# 3rd Convolutional Layer
alexnet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))

# 4th Convolutional Layer
alexnet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))

# 5th Convolutional Layer
alexnet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
alexnet.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# Passing it to a Fully Connected layer
alexnet.add(Flatten())

# 1st Fully Connected Layer
alexnet.add(Dense(4096, activation='relu'))
alexnet.add(Dropout(0.5))

# 2nd Fully Connected Layer
alexnet.add(Dense(4096, activation='relu'))
alexnet.add(Dropout(0.5))

# Output Layer
alexnet.add(Dense(10, activation='softmax'))

In [ ]:
alexnet.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
alexnet.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))